In [1]:
import os
from tqdm.auto import tqdm
from evaluate import load
from utils import TASTEset
import pandas as pd
import json

data_repo = 'pgajo/mdeberta_GZ-GOLD-NER-ALIGN_105_U1_S0_DROP0'
data = TASTEset.from_datasetdict(data_repo, keep_raw = True)

ew_taste_path = '/home/pgajo/working/food/data/TASTEset/data/EW-TASTE/EW-TASTE_en-it_DEEPL_LOC.json'

with open(ew_taste_path, 'r', encoding='utf8') as f:
    train_data = json.load(f)

/home/pgajo/working/food/.o/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dict = {}

for line in train_data['annotations']:
    for i in range(len(line['ents_en'])):
        ent_en = line['text_en'][line['ents_en'][i][0]:line['ents_en'][i][1]]
        ent_it = line['text_it'][line['ents_it'][i][0]:line['ents_it'][i][1]]
        train_dict[ent_en.lower()] = ent_it.lower()

In [3]:
from evaluate import load
squad_metric_evaluate = load("squad_v2")

In [4]:
import re
import uuid
preds = []
trues = []
en_pattern = re.compile(r'(?<=\•\s).*(?=\s\•)')
for el in data['test']:
    # print(data['test'][0]['context'].lower())
    id = str(uuid.uuid4())
    en = re.search(en_pattern, el['query']).group()
    try:
        it_pattern = re.compile(train_dict[en.lower()])
        # print(en.lower(), it_pattern)
        answer = re.search(it_pattern, data['test'][0]['context'].lower())
        if answer is None:
            answer = ''
        else:
            answer = answer.group()
    except KeyError:
        answer = ''
    
    dict_pred = {
            'prediction_text': answer,
            'id': id,
            'no_answer_probability': 0
        }
    preds.append(dict_pred)
    dict_true = {
    'answers': {
        'answer_start': [int(el['answer_start'])],
        'text': [el['answer'].lower()],
        },
        'id': id
    }
    trues.append(dict_true)

metrics_dict = squad_metric_evaluate.compute(predictions=preds, references=trues)
metrics_dict

{'exact': 27.149777957206297,
 'f1': 27.25070649979814,
 'total': 4954,
 'HasAns_exact': 27.149777957206297,
 'HasAns_f1': 27.25070649979814,
 'HasAns_total': 4954,
 'best_exact': 27.149777957206297,
 'best_exact_thresh': 0.0,
 'best_f1': 27.25070649979814,
 'best_f1_thresh': 0.0}

In [8]:
results_path = '/home/pgajo/working/food/results'
model_results_path = os.path.join(results_path, data_repo.split('/')[-1])
if not os.path.isdir(model_results_path):
    os.makedirs(model_results_path)
import pandas as pd

df_metrics = pd.DataFrame(metrics_dict, ['Value']).T
model_name = 'TASTEset_dict_model'
df_metrics.to_csv(os.path.join(model_results_path, model_name))

In [ ]:
import re
en_pattern_raw = r'(?<=\•\s).*(?=\s\•)'
en_pattern = re.compile(en_pattern_raw)
en = re.search(en_pattern, data['test'][0]['query']).group()
print(en)
it = train_dict[en.lower()]
it_pattern = re.compile(it)
answer = re.search(it_pattern, data['test'][0]['context'].lower())
answer


In [ ]:
from evaluate import load
squad_metric_evaluate = load("squad_v2")
squad_metric_evaluate.compute(predictions=self.preds[split], references=self.trues[split])['f1']

In [ ]:
def get_eval_batch(self, model_outputs, batch, split):
        self.split = split
        start_preds = torch.argmax(model_outputs['start_logits'], dim=1)
        end_preds = torch.argmax(model_outputs['end_logits'], dim=1)
        pred_batch = [el for el in zip(start_preds.tolist(),
                                       end_preds.tolist())]
        true_batch = [el for el in zip(batch["start_positions"].tolist(),
                                       batch["end_positions"].tolist())]
        pred_batch_ids = [str(uuid.uuid4()) for i in range(len(start_preds))]

        for i, pair in enumerate(pred_batch):
            if pair[0] >= pair[1]:
                text_pred = ''
            else:
                text_pred = self.tokenizer.decode(batch['input_ids'][i][pair[0]:pair[1]])
                if not isinstance(text_pred, str):
                    text_pred = ''
            dict_pred = {
                'prediction_text': text_pred,
                'id': pred_batch_ids[i],
                'no_answer_probability': 0
            }

        for i, pair in enumerate(true_batch):
            text_true = self.tokenizer.decode(
                batch['input_ids'][i][pair[0]:pair[1]])
            dict_true = {
                'answers': {
                    'answer_start': [true_batch[0][0]],
                    'text': [text_true],
                    },
                    'id': pred_batch_ids[i]
                }

        self.preds[split].append(dict_pred)
        self.trues[split].append(dict_true)

In [ ]:
for line in data:
    